In [2]:
pip install transformers torch


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Nuwanga Wijamuni\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [12]:
!pip uninstall accelerate
!pip install accelerate==0.20.1



^C
     -------------------------------------- 227.5/227.5 KB 7.2 kB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


You should consider upgrading via the 'C:\Users\Nuwanga Wijamuni\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [10]:
import pandas as pd
import numpy as np
import re
from transformers import (TextDataset, DataCollatorForLanguageModeling, 
                          GPT2Tokenizer, GPT2LMHeadModel, 
                          Trainer, TrainingArguments)

# Cleaning function
def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    return s

# Load the dataset
df = pd.read_excel("Allergy.xlsx",header=0) 
df = df.dropna()
text_data = open('Allergy.txt', 'w')
for idx, item in df.iterrows():
  conversation = cleaning(item["Patient"] + " " + item["Doctor"])
  text_data.write(conversation)
text_data.close()

# Helper functions for training
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=False
    )
    return data_collator

# Training function
def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):

  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)
  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)
  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

# Set parameters 
train_file_path = "Allergy.txt"
model_name = 'gpt2'
output_dir = 'result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

# Start training
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

# Text generation
def generate_text(sequence, max_length):
    model_path = "result"
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

# Get user input
sequence = input("Please provide a seed sequence: ") 
while True:
    try:
        max_len = int(input("Please provide the max length for the output (integer): ")) 
        break
    except ValueError:
        print("Please enter a valid integer for max length.")

generate_text(sequence, max_len)


c:\Users\Nuwanga Wijamuni\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Nuwanga Wijamuni\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\Nuwanga Wijamuni\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 1.2/1.2 MB 519.5 kB/s eta 0:00:00
     ------------------------------------ 172.3/172.3 MB 556.7 kB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 589.9 kB/s eta 0:00:00
     ------------------------------------ 536.2/536.2 KB 623.2 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
